In [1]:
import numpy as np
from numpy.testing import assert_almost_equal as aae
from numpy.testing import assert_equal as ae

In [2]:
from gravmag import inverse_distance as idist
from gravmag import convolve as conv

In [3]:
def transpose_BTTB(BTTB):
    """
    Return the data structure for the transposed BTTB matrix.
    This function if valid only for symmetries 'symm' and 'skew'.
    """

    columns = np.copy(BTTB["columns"])

    if BTTB["symmetry_structure"] == "symm":
        if BTTB["symmetry_blocks"] == "symm":
            pass
        else: # BTTB["symmetry_blocks"] == "skew"
            columns[:,1:] *= -1
    else: # BTTB["symmetry_structure"] == "skew"
        if BTTB["symmetry_blocks"] == "symm":
            columns[1:,:] *= -1
        else: # BTTB["symmetry_blocks"] == "skew"
            columns[:,1:] *= -1
            columns[1:,:] *= -1

    BTTB_T = {
        "symmetry_structure" : BTTB["symmetry_structure"],
        "symmetry_blocks" : BTTB["symmetry_blocks"],
        "nblocks": BTTB["nblocks"],
        "columns": columns,
        "rows": None,
    }

    return BTTB_T

In [4]:
grid_yx = {
    'x' : np.arange(0, 8000, 200, dtype=float)[:,np.newaxis],
    'y' : np.arange(0, 12000, 80, dtype=float),
    'z' : 0.,
    'ordering' : 'yx'
}

In [5]:
h=500

In [6]:
R2 = idist.sedm_BTTB(data_grid=grid_yx, delta_z=h)

In [7]:
tensor = idist.grad_tensor_BTTB(
    data_grid=grid_yx,
    delta_z=h,
    SEDM=R2,
    components=["xx", "xy", "xz", "yy", "yz", "zz"],
)

In [8]:
symmetry_structure = [
    'symm', 'skew', 'skew',
    'symm', 'symm',
    'symm'
]

symmetry_blocks = [
    'symm', 'skew', 'symm',
    'symm', 'skew',
    'symm'
]

In [9]:
for (component, structure, blocks) in zip(tensor, symmetry_structure, symmetry_blocks):
    # compute the full BTTB matrix and its transposed
    BTTB = {
        "symmetry_structure": structure,
        "symmetry_blocks": blocks,
        "nblocks": grid_yx['x'].size,
        "columns": np.reshape(
            a=component, newshape=(grid_yx['x'].size, grid_yx['y'].size)
        ),
        "rows": None,
    }
    BTTB_full = conv.generic_BTTB(BTTB=BTTB)
    BTTB_T = transpose_BTTB(BTTB)
    BTTB_T_full = conv.generic_BTTB(BTTB=BTTB_T)
    ae(BTTB_full.T, BTTB_T_full)
    # compute the eigenvalues
    L = conv.eigenvalues_BCCB(BTTB=BTTB, ordering="row")
    L_T = conv.eigenvalues_BCCB(BTTB=BTTB_T, ordering="row")
    aae(np.conj(L), L_T, decimal=12)